In [8]:
# Instalar dependencias necesarias
!pip install pandas numpy matplotlib seaborn scikit-learn openpyxl

# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, silhouette_score

# Cargar los datos
file_path = "RIJA _ Iniciativas de Participacion Ciu dadana en el Poder Judicial - Mapeo.xlsx"
data = pd.read_excel(file_path, sheet_name='Mapeo de Casos')

# Limpieza preliminar de datos
data.dropna(how='all', inplace=True)  # Eliminar filas que están completamente vacías
data.fillna(method='ffill', inplace=True)  # Rellenar valores faltantes hacia adelante donde sea apropiado

# Mostrar información básica sobre el conjunto de datos
print(data.info())
print(data.head())

# Análisis Estadístico Básico
print(data.describe())

# Distribución de datos categóricos
categorical_columns = ['Región', 'País', 'Tipo de Institución', 'Nivel de Participación (Manual CEPAL)']

for column in categorical_columns:
    print(f"Distribución de {column}:")
    print(data[column].value_counts())
    plt.figure(figsize=(10, 5))
    sns.countplot(x=column, data=data)
    plt.xticks(rotation=45)
    plt.show()

# Matriz de correlación
numeric_data = data.select_dtypes(include=[np.number])  # Seleccionar columnas numéricas
correlation_matrix = numeric_data.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

# Ingeniería de características
data['Initiative_Length'] = data['Iniciativa'].apply(len)
data['Year'] = pd.to_datetime(data['Año de Inicio'], errors='coerce').dt.year
data['Initiative_Length'].fillna(data['Initiative_Length'].median(), inplace=True)
data['Year'].fillna(data['Year'].mode()[0], inplace=True)

# Verificar que no haya valores nulos en las características seleccionadas
print(data[['Initiative_Length', 'Year']].isnull().sum())

# Estandarización de datos
features = ['Initiative_Length', 'Year']
x = StandardScaler().fit_transform(data[features])

# Aplicación de PCA
pca = PCA(n_components=2)  # Reducir a dos componentes principales
principal_components = pca.fit_transform(x)
pca_df = pd.DataFrame(data=principal_components, columns=['CP1', 'CP2'])

# Graficar los dos primeros componentes principales
plt.figure(figsize=(8, 6))
sns.scatterplot(x='CP1', y='CP2', hue=data['Tipo de Institución'], data=pca_df, palette='Set1')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.title('PCA - Primeros dos componentes principales')
plt.grid(True)
plt.show()

# Varianza explicada por cada componente
print(f"Varianza explicada por componentes PCA: {pca.explained_variance_ratio_}")

# Clustering en los componentes principales
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(pca_df)

# Agregar resultados del clustering al DataFrame
pca_df['Cluster'] = clusters

# Graficar los clusters
plt.figure(figsize=(8, 6))
sns.scatterplot(x='CP1', y='CP2', hue='Cluster', data=pca_df, palette='viridis')
plt.title('Clustering K-means en resultados de PCA')
plt.legend(title='Cluster')
plt.show()

# Puntaje de Silhouette
score = silhouette_score(pca_df[['CP1', 'CP2']], pca_df['Cluster'])
print(f"Puntaje de Silhouette: {score}")

# Preparar datos para aprendizaje supervisado
X = data[features]
y = data['Tipo de Institución']

# División de datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar un Clasificador RandomForest
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predicciones y evaluación
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Importancia de características
importances = clf.feature_importances_
feature_importance_df = pd.DataFrame({'Feature': features, 'Importance': importances})
print(feature_importance_df)

# Visualización de la importancia de las características
plt.figure(figsize=(10, 6))
sns.barplot(x='Feature', y='Importance', data=feature_importance_df)
plt.title('Importancia de las características')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'G:\\Mi unidad\\Proyecto Data Visualization - RIJA\x02-Salida\x01-Análisis Exploratorio de los Datos\\RIJA _ Iniciativas de Participacion Ciu dadana en el Poder Judicial - Mapeo.xlsx'